In [118]:
import pandas as pd
import numpy as np
from HEI_2015_Scoring import hei2015

In [119]:
#load and clean FFQ data
spain_data = pd.read_excel('/projects/thdmi/metadata/nutrient_ffq_data/Spain FFQ wt corrected gr of food.xlsx', index_col=0)
spain_data.drop(columns=['Columna1'], inplace=True)
spain_data.index = ['10317.' + i for i in spain_data.index]
spain_data.columns = [c.strip() for c in spain_data.columns]
# upload metadata
md = pd.read_csv('/home/lakhatib/thdmi/pangenome_filtered/data/thdmi_metadata_valid_covariates.tsv', sep = '\t', index_col=0, low_memory=False)
spain_data = spain_data.loc[spain_data.index.intersection(md.index)]

In [120]:
#Load and clean nutrient data
spain_nutrients = pd.read_excel('/projects/thdmi/metadata/nutrient_ffq_data/spain_nutrients.xlsx', index_col=0)
spain_nutrients.index = ['10317.' + i for i in spain_nutrients.index]
spain_nutrients.columns = [c.strip() for c in spain_nutrients.columns]
spain_nutrients = spain_nutrients.loc[spain_nutrients.index.intersection(md.index)]

In [121]:
#Load and MPED Conversions
MPED_conversions = pd.read_excel('/home/lakhatib/thdmi/pangenome_filtered/data/HEI2015- Spain.xlsx', sheet_name=2)
MPED_conversions.columns = [c.strip() for c in MPED_conversions.columns]
mped_mapping = MPED_conversions.set_index('Food item (FFQ Spain)').filter(like='MPED_')
mped_mapping.index = [c.strip() for c in mped_mapping.index]

In [122]:
#Convert values from 100 grams to 1 gram
mped_mapping = mped_mapping / 100

In [123]:
#Align MPED mappings with Spain data and filter foods
common_foods = spain_data.columns.intersection(mped_mapping.index)
spain_data_filtered = spain_data[common_foods]
mped_mapping_filtered = mped_mapping.loc[common_foods]

In [124]:
#Ensure numeric values 
spain_data_filtered = spain_data_filtered.apply(pd.to_numeric, errors='coerce')
mped_mapping_filtered = mped_mapping_filtered.apply(pd.to_numeric, errors='coerce')

# Convert dataframes to NumPy arrays
mped_mapping_array = mped_mapping_filtered.values  # Shape: (n_foods, n_components)
spain_data_array = spain_data_filtered.values  # Shape: (n_participants, n_foods)

# Perform matrix multiplication
mped_scores = np.dot(spain_data_array, mped_mapping_array)

# Create a DataFrame with participants as rows and MPED components as columns
mped_df = pd.DataFrame(mped_scores, 
                              index=spain_data_filtered.index, 
                              columns=mped_mapping_filtered.columns)

In [125]:
spain_data_filtered = spain_data_filtered.replace(np.nan, 0)

In [126]:
#Add nutrient information to dataframe
mped_df[spain_nutrients.columns] = spain_nutrients[spain_nutrients.columns]

In [127]:
# Calculate monopoly (sum of monounsaturated and polyunsaturated fats)
mped_df['monopoly'] = mped_df['Monounsaturated fats (MUFA) (g)'] + mped_df['Polyunsaturated fats (PUFA) (g)']

In [128]:
# Combine vegetable intake with legumes
mped_df['VTOTALLEG'] = mped_df['MPED_V_TOTAL'] + mped_df['MPED_V_LEGUMES']
mped_df['VDRKGRLEG'] = mped_df['MPED_V_DRKGR'] + mped_df['MPED_V_LEGUMES']

In [129]:
# Convert cup equivalents of legumes to ounce equivalents
mped_df['protlegumes'] = mped_df['MPED_V_LEGUMES'] * 4

In [130]:
# Calculate total protein intake from various sources
mped_df['PFALLPROTLEG'] = (
    mped_df['MPED_PF_TOTAL'] + mped_df['protlegumes']
)
mped_df['PFSEAPLANTLEG'] = (
    mped_df['MPED_PF_SEAFD_HI'] + mped_df['MPED_PF_SEAFD_LOW'] + 
    mped_df['MPED_PF_SOY'] + mped_df['MPED_PF_NUTSDS'] + mped_df['protlegumes']
)

In [131]:
#calculate HEI

hei_mped_df = hei2015(mped_df, mped_df['Energy(Kcal)'], mped_df['VTOTALLEG'], mped_df['VDRKGRLEG'], mped_df['MPED_F_TOTAL'], mped_df['MPED_F_CITMLB'], mped_df['MPED_G_WHOLE'], 
                 mped_df['MPED_D_TOTAL'], mped_df['PFALLPROTLEG'], mped_df['PFSEAPLANTLEG'], mped_df['monopoly'], mped_df['Saturated fats (SFA) (g)'], 
                 mped_df['Sodium (mg)'], mped_df['MPED_G_REFINED'], mped_df['MPED_ADD_SUGARS'])

In [132]:
hei_df = hei_mped_df.filter(like='HEI')

In [133]:
hei_df.describe()

,HEI2015C1_TOTALVEG,HEI2015C2_GREEN_AND_BEAN,HEI2015C3_TOTALFRUIT,HEI2015C4_WHOLEFRUIT,HEI2015C5_WHOLEGRAIN,HEI2015C6_TOTALDAIRY,HEI2015C7_TOTPROT,HEI2015C8_SEAPLANT_PROT,HEI2015C9_FATTYACID,HEI2015C10_SODIUM,HEI2015C11_REFINEDGRAIN,HEI2015C12_SFAT,HEI2015C13_ADDSUG,HEI2015_TOTAL_SCORE
count,427.000000,427.000000,427.000000,427.000000,427.000000,427.000000,427.000000,427.000000,428.000000,428.000000,427.000000,428.000000,427.000000,428.000000
mean,4.886380,4.764228,4.017631,2.821210,3.187920,5.359959,4.944333,4.948410,8.022321,7.298522,9.794848,4.936393,9.729837,74.584760
std,0.445357,0.819769,1.408075,1.743460,2.965238,3.056231,0.393579,0.416267,2.447814,2.820894,1.013749,2.822238,0.822033,8.622239
min,1.053009,0.010053,0.030630,0.000000,0.000000,0.000000,0.440174,0.581168,0.000000,0.000000,0.000000,0.000000,2.062523,28.749742
25%,5.000000,5.000000,3.157896,1.250432,0.738175,2.986587,5.000000,5.000000,6.461602,5.738591,10.000000,2.879589,10.000000,69.360841
50%,5.000000,5.000000,5.000000,2.701318,2.195305,5.305545,5.000000,5.000000,9.379448,8.008762,10.000000,4.756087,10.000000,75.661624
75%,5.000000,5.000000,5.000000,5.000000,5.239653,7.652704,5.000000,5.000000,10.000000,10.000000,10.000000,6.854984,10.000000,80.781362
max,5.000000,5.000000,5.000000,5.000000,10.000000,10.000000,5.000000,5.000000,10.000000,10.000000,10.000000,10.000000,10.000000,93.500124


In [136]:
hei_df = hei_df.drop(index=['10317.DM2898999'])

In [137]:
hei_df['HEI2015_TOTAL_SCORE'].describe()

count    427.000000
mean      74.692102
std        8.341128
min       38.849027
25%       69.409830
50%       75.674456
75%       80.784713
max       93.500124
Name: HEI2015_TOTAL_SCORE, dtype: float64

In [139]:
hei_df.to_csv('/home/lakhatib/thdmi/pangenome_filtered/data/spain_HEI_df.csv')